#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader('speech.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)

In [2]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [5]:
embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
db = FAISS.from_documents(docs, embeddings)
db

In [6]:
### Qurying the vector store
query = "How does the speaker describe the desired outcome of the war?"
docs = db.similarity_search(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [7]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [9]:
docs_and_score_db = db.similarity_search_with_score(query)
docs_and_score_db

[(Document(id='9a68f4b5-9f73-4bf0-b6f1-59b1bf79e278', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
  np.float32(0.38339365)),
 (Document(id='d756eac9-4779-41bc-8677-cd9b4c3e5789', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we

In [10]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[-0.00260819960385561,
 0.004657368175685406,
 0.026616821065545082,
 -0.09536195546388626,
 -0.017837049439549446,
 0.012348448857665062,
 0.00046351144555956125,
 0.003065569559112191,
 -0.0020710884127765894,
 -0.004327680915594101,
 0.006557676941156387,
 -0.004533109720796347,
 0.01949237659573555,
 6.31582661299035e-05,
 0.14492380619049072,
 -0.005647031124681234,
 0.00409280601888895,
 7.869857654441148e-05,
 0.02282215654850006,
 0.019017037004232407,
 0.019770968705415726,
 0.003352290950715542,
 0.01986098475754261,
 -0.020881935954093933,
 0.015104594640433788,
 -0.004185147117823362,
 0.003941291943192482,
 0.0176137313246727,
 0.019281355664134026,
 -0.011442931368947029,
 0.0060529629699885845,
 0.0023356350138783455,
 0.0008626870112493634,
 0.010043267160654068,
 0.006173618603497744,
 0.014410206116735935,
 0.007703773211687803,
 -0.021953685209155083,
 0.006266090553253889,
 0.0070595466531813145,
 -0.02408169023692608,
 -0.0014989738119766116,
 -0.001857090042904019

In [11]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='9a68f4b5-9f73-4bf0-b6f1-59b1bf79e278', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='d756eac9-4779-41bc-8677-cd9b4c3e5789', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things w

In [12]:
### Saving And Loading
db.save_local("faiss_index")

In [13]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [14]:
docs

[Document(id='9a68f4b5-9f73-4bf0-b6f1-59b1bf79e278', metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her blood and her might for the principles that gave her birth and happiness and the peace which she has treasured. God helping her, she can do no other.'),
 Document(id='d756eac9-4779-41bc-8677-cd9b4c3e5789', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things w